# XỬ LÝ DỮ LIỆU OLD DATA
Mục tiêu của notebook này là xử lý dữ liệu có sẵn (old data) đã thu thập từ Google Data Search.

Dữ liệu thu thập được bao gồm:   
Timestamp: Thời gian giao dịch.  
Open: Giá mở cửa của cặp tiền điện tử.  
High: Giá cao nhất trong ngày.  
Low: Giá thấp nhất trong ngày.  
Close: Giá đóng cửa của cặp tiền điện tử.  
Volume: Khối lượng giao dịch trong ngày.  
Pair: Tên cặp tiền điện tử đã thực hiện giao dịch.  
Exchange: Tên sàn giao dịch đã thực hiện giao dịch.

Chúng ta sẽ làm sạch, xử lý giá trị thiếu, tạo đặc trưng, và xử lý cân mất cân bằng dữ liệu.


## CẤU TRÚC NOTEBOOK
1. **Đọc dữ liệu**:  
   Tải dữ liệu từ nguồn hoặc file đã lưu.
2. **Phân tích dữ liệu**:  
   Vẽ các biểu đồ phân tích.
3. **Làm sạch dữ liệu**:  
   Xử lý giá trị thiếu, trùng lặp và chuyển đổi định dạng.
4. **Tạo đặc trưng (Feature Engineering)**:  
   Tạo thêm các chỉ số kỹ thuật.
5. **Cân bằng dữ liệu**:  
   Dùng SMOTE để cân bằng tập dữ liệu.
6. **Lưu dữ liệu đã xử lý**:  
   Lưu kết quả để sử dụng cho các bước tiếp theo.


#### Import các thư viện

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Đường dẫn thư mục

In [ ]:
# Đường dẫn thư mục
data_dir = "../data/old_data/"
processed_dir = os.path.join(data_dir, "processed")
plot_dir = "../plots/data_analysis"

# Tạo thư mục lưu trữ dữ liệu nếu chưa có
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

# Danh sách các sàn giao dịch
exchanges = ["binance", "bittrex", "kraken", "kucoin", "lbank"]

### Đọc dữ liệu
Chúng ta sử dụng thư viện pandas để đọc dữ liệu từ thư mục `data/raw/old_data`.

Sau đó hợp nhất dữ liệu.


In [ ]:
# Hợp nhất dữ liệu từ các file CSV
all_data = []

for exchange in exchanges:
    exchange_dir = os.path.join(data_dir, "raw", exchange)
    files = [f for f in os.listdir(exchange_dir) if f.endswith(".csv")]

    for file in files:
        file_path = os.path.join(exchange_dir, file)
        df = pd.read_csv(file_path)
        
        # Lấy thông tin cặp giao dịch từ tên file
        pair = file.split("_")[1]
        
        # Thêm cột thông tin
        df["Pair"] = pair
        df["Exchange"] = exchange
        all_data.append(df)

# Hợp nhất toàn bộ dữ liệu
data = pd.concat(all_data, ignore_index=True)
print("Dữ liệu ban đầu:")
print(data.head())


In [ ]:
# Lưu dữ liệu đã xử lý
processed_file = os.path.join(processed_dir, "data_concated.csv")
data.to_csv(processed_file, index=False)
print(f"Dữ liệu đã lưu tại: {processed_file}")

### Phân Tích Dữ Liệu

#### Biểu đồ phân phối khối lượng giao dịch (Volume)

In [ ]:
# Vẽ biểu đồ phân phối Volume
plt.figure(figsize=(10, 6))
sns.histplot(data["Volume"], bins=50, kde=True, color='blue')
plt.title("Phân Phối Khối Lượng Giao Dịch (Volume)")
plt.xlabel("Volume")
plt.ylabel("Tần suất")
plt.savefig(os.path.join(plot_dir, "volume_distribution.png"))
plt.show()

#### Biểu đồ biến động giá (Price Volatility)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data["Timestamp"], data["High"], label='Giá Cao Nhất (High)', color='green')
plt.plot(data["Timestamp"], data["Low"], label='Giá Thấp Nhất (Low)', color='red')
plt.title("Biến Động Giá Theo Thời Gian")
plt.xlabel("Thời gian")
plt.ylabel("Giá")
plt.legend()
plt.savefig(os.path.join(plot_dir, "price_volatility.png"))
plt.show()


#### Ma trận tương quan giữa các đặc trưng

In [ ]:
plt.figure(figsize=(8, 6))
correlation = df[["Open", "High", "Low", "Close", "Volume"]].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Ma Trận Tương Quan Giữa Các Đặc Trưng")
plt.savefig(os.path.join(plot_dir, "correlation_matrix.png"))
plt.show()


### Làm sạch dữ liệu
Các bước bao gồm:  
  Xóa dữ liệu trùng lặp.  
  Loại bỏ các giao dịch không hợp lệ (giá trị âm, null).  
  Chuyển đổi định dạng thời gian `Timestamp` sang datetime.  


In [ ]:
# Chuyển Timestamp sang datetime
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

# Loại bỏ giá trị trùng lặp
data.drop_duplicates(subset=["Timestamp", "Pair", "Exchange"], inplace=True)

# Sắp xếp dữ liệu
data.sort_values(by="Timestamp", inplace=True)
print(data.info())


#### Điền giá trị thiếu bằng phương pháp Linear Interpolation
Phương pháp Linear Interpolation là một kỹ thuật để ước lượng và thay thế các giá trị bị thiếu (NaN) trong dữ liệu dựa trên các giá trị liền kề. 

In [ ]:
# Kiểm tra giá trị thiếu
missing_values = data.isnull().sum()
print("Giá trị thiếu:\n", missing_values)

# Điền giá trị thiếu bằng phương pháp Linear Interpolation
data.interpolate(method="linear", inplace=True)

# Xác nhận lại không còn giá trị thiếu
print("Giá trị thiếu after sau khi điền:\n", data.isnull().sum())


### Tạo đặc trưng (Feature Engineering)
Chúng ta thêm các đặc trưng mới để hỗ trợ mô hình học máy:
- **Tăng đột biến khối lượng (Volume Spike)**.
- **Tỷ lệ thay đổi giá (Price Change %)**.
- **Biến động giá (Volatility)**.


In [ ]:
# Tính toán các đặc trưng mới
data["Price_Change_Pct"] = (data["Close"] - data["Open"]) / data["Open"] * 100
data["Volume_Spike"] = data["Volume"] / data["Volume"].rolling(window=10).mean()
data["Volatility"] = data["High"] - data["Low"]

# Điền NaN trong cột mới (do rolling tạo ra)
data.fillna(0, inplace=True)


print("Dữ liệu với đặc trưng mới:")
print(data[["Price_Change_Pct", "Volume_Spike", "Volatility"]].head())


### Chuẩn hoá dữ liệu
 

Chuẩn hoá các cột cần thiết bằng MinMaxScaler.

In [ ]:
# Chuẩn hóa các cột
scaler = MinMaxScaler()

# Chuyển đổi Timestamp sang số giây kể từ 1970-01-01
data["Timestamp"] = (data["Timestamp"] - pd.Timestamp("2018-4-20")) / pd.Timedelta(seconds=1)

columns_to_scale = ["Open", "High", "Low", "Close", "Volume"]
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

print("Dữ liệu sau chuẩn hóa:")
print(data.head())

### Dán nhãn dữ liệu

Dán nhãn cho dữ liệu gồm 3 bước:
- Dán nhãn dựa trên quy tắc: Price_Change_Pct, Volume_Spike, Volatility.
- Dán nhãn dựa trên điểm bất thường: bằng Isolation Forest
- Dán nhãn bẳng phân cụm

#### Hàm dán nhãn theo quy tắc

In [ ]:
def statistical_rules(df):
    # Tính các ngưỡng thống kê
    price_change_threshold = df['Price_Change_Pct'].quantile(0.95)
    volume_spike_threshold = df['Volume_Spike'].quantile(0.95)
    volatility_threshold = df['Volatility'].quantile(0.95)

    # Áp dụng quy tắc vectorized
    price_anomaly = (abs(df['Price_Change_Pct']) > price_change_threshold)
    volume_anomaly = (df['Volume_Spike'] > volume_spike_threshold)
    volatility_anomaly = (df['Volatility'] > volatility_threshold)

    # Kết hợp các điều kiện
    return ((price_anomaly & volume_anomaly) |
            (price_anomaly & volatility_anomaly) |
            (volume_anomaly & volatility_anomaly)).astype(int)


#### Hàm dán nhãn bằng Isolation Forest

In [ ]:
# Bước 2: Phát hiện bất thường bằng Isolation Forest với xử lý batch
def isolation_forest_batch(df, batch_size=10000):
    features = ['Open', 'High', 'Low', 'Close', 'Volume',
               'Price_Change_Pct', 'Volume_Spike', 'Volatility']

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])

    labels = np.zeros(len(df))

    # Xử lý từng batch
    for i in range(0, len(df), batch_size):
        batch = scaled_features[i:i+batch_size]
        iso_forest = IsolationForest(contamination=0.01, random_state=42)
        batch_labels = (iso_forest.fit_predict(batch) == -1).astype(int)
        labels[i:i+batch_size] = batch_labels

    return labels


#### Hàm dán nhãn bằng Clustering


In [ ]:
# Bước 3: Phân cụm để phát hiện mẫu bất thường
def cluster_anomalies(df, n_clusters=50):
    features = ['Open', 'High', 'Low', 'Close', 'Volume',
               'Price_Change_Pct', 'Volume_Spike', 'Volatility']

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])

    # Sử dụng MiniBatchKMeans cho hiệu năng tốt hơn
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(scaled_features)

    # Đánh dấu các cụm nhỏ là bất thường
    cluster_sizes = np.bincount(cluster_labels)
    small_clusters = cluster_sizes < np.percentile(cluster_sizes, 10)
    return np.isin(cluster_labels, np.where(small_clusters)[0]).astype(int)


In [ ]:
# Thực hiện 3 hàm trên
labels_stats = statistical_rules(data)
labels_iforest = isolation_forest_batch(data)
labels_cluster = cluster_anomalies(data)

In [ ]:
# Kết hợp kết quả từ 3 phương pháp
final_labels = ((labels_stats + labels_iforest + labels_cluster) >= 2).astype(int)
data["Label"] = final_labels

### Kiểm tra lại dữ liệu sau khi dán nhãn

#### Kiểm tra phân phối nhãn

In [ ]:
# Kiểm tra phân phối nhãn
label_distribution = data["Label"].value_counts()
print("Phân phối nhãn:\n", label_distribution)

# Vẽ biểu đồ phân phối nhãn
plt.figure(figsize=(6, 4))
sns.barplot(x=label_distribution.index, y=label_distribution.values, palette="coolwarm")
plt.title("Phân Phối Nhãn Trong Dữ Liệu")
plt.xlabel("Nhãn (Label)")
plt.ylabel("Số lượng mẫu")
plt.savefig(os.path.join(plot_dir, "label_distribution.png"))
plt.show()

#### Phân tích tương quan sau khi dán nhãn

In [ ]:
correlation = data[["Timestamp", "Open", "High", "Low", "Close", "Volume", "Price_Change_Pct", "Volume_Spike", "Volatility", "Label"]].corr()

# Vẽ heatmap tương quan
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Ma Trận Tương Quan Giữa Các Đặc Trưng Và Nhãn")
plt.savefig(os.path.join(plot_dir, "correlation_matrix_label.png"))
plt.show()

### Lưu dữ liệu
Dữ liệu sau khi xử lý được lưu vào file `data/processed/data_processed.csv`.


In [ ]:
# Lưu dữ liệu đã xử lý
processed_file = os.path.join(processed_dir, "data_processed.csv")
data.to_csv(processed_file, index=False)
print(f"Dữ liệu đã lưu tại: {processed_file}")
